In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder 
#from xgboost import XGBClassifier

In [2]:
input_file=open('train.json','r')
train=json.load(input_file)
input_file=open('test.json','r')
test=json.load(input_file)
train[0]
len(test)

9944

In [3]:
for i in range(39774):
     a=train[i]['ingredients']
     new=[]
     for ingredients in a:
         new.append('_'.join((ingredients.split(' '))))
     train[i]['ingredients']=new
for i in range(9944):
     a=test[i]['ingredients']
     new=[]
     for ingredients in a:
         new.append('_'.join((ingredients.split(' '))))
     test[i]['ingredients']=new

In [4]:
train=pd.DataFrame.from_dict(train)
train=train[['id','cuisine','ingredients']]
test=pd.DataFrame.from_dict(test)
test=test[['id','ingredients']]


In [5]:
tfidf = TfidfVectorizer(min_df=3)
train['ingredients']=[" ".join(ingredient) for ingredient in train['ingredients'].values]
X_train = tfidf.fit_transform(train['ingredients'])
X_train.shape

(39774, 4255)

In [6]:
test['ingredients']=[" ".join(ingredient) for ingredient in test['ingredients'].values]
X_test = tfidf.fit_transform(test['ingredients'])
X_test.shape

(9944, 2450)

In [7]:
svd = TruncatedSVD(n_components=2450)

In [8]:
X_proj = svd.fit_transform(X_train)
#X_proj_test = svd.fit_transform(X_test)


In [9]:
np.cumsum(svd.explained_variance_ratio_)[-1]
#it returns a cumulative sum of the variance explained by the first (i+1) dimensions

0.94827115123566252

In [10]:
import time
model_svc = LinearSVC()
#0.78851
#model_rfc = RandomForestClassifier(max_depth=25, random_state=0)
#model = LogisticRegression()
#model_gbc=  GradientBoostingClassifier()
#model = XGBClassifier()
#cross_val_score(model_gbc,X_proj,train['cuisine'])
#cross_val_score(model_rfc,X_proj,train['cuisine'])
#cross_val_score(model_svc,X_proj,train['cuisine'])


t1=time.time()

model_svc.fit(X_proj,train['cuisine'])
#print(cross_val_score(model_svc,X_proj,train['cuisine']))
t2=time.time()
print(t2-t1)

85.5200309753418


In [13]:
a=model_svc.predict(X_test)
len(a)

9944

In [14]:
test_copy=test
test_copy['cuisine']=a
test_copy = test_copy.sort_values(by='id' , ascending=True)
test_copy[['id' , 'cuisine' ]].to_csv('linear_svc_submission_svd_2400.csv',index=False)

In [ ]:
#Score of -0.78851 on Kaggle